In [ ]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv
import json
import matplotlib.pyplot as plt

load_dotenv()

In [ ]:
API_KEY = os.getenv('ODDS_API_KEY')
BASE_URL = os.getenv('ODDS_API_BASE_URL')

In [ ]:
odds_api_get_sports = f'{BASE_URL}/v4/sports/?apiKey={API_KEY}'

r = requests.get(odds_api_get_sports)

In [ ]:
sport_keys = []
for sport in r.json():
    
    print(sport['key'])
    sport_keys.append(sport['key'])

In [ ]:
t_sport_keys = [sk for sk in sport_keys if sk.startswith('mma')]
t_sport_keys

In [ ]:
odds_api_get_odds = f'{BASE_URL}/v4/sports/{t_sport_keys[0]}/odds/?apiKey={API_KEY}&regions=us,us2&markets=h2h,spreads,totals'
odds_api_get_odds

In [ ]:
r2 = requests.get(odds_api_get_odds)

r2.json()

In [ ]:
for card in r2.json():
    print(card)
    print('========================')

In [ ]:
from datetime import datetime

today_str = datetime.today().strftime('%Y-%m-%d')

r2_text = json.dumps(r2.json())
with open(f'./odds_data/{today_str}_{t_sport_keys[0]}_odds.txt','w') as outfile:
    outfile.write(r2_text)

In [ ]:
data = r2.json()

In [ ]:
rows = []
for event in data:
    for bookmaker in event["bookmakers"]:
        for market in bookmaker["markets"]:
            if market["key"] == "h2h":
                for outcome in market["outcomes"]:
                    rows.append({
                        "fight": f"{event['home_team']} vs {event['away_team']}",
                        "commence_time": event["commence_time"],
                        "bookmaker": bookmaker["title"],
                        "fighter": outcome["name"],
                        "decimal_odds": outcome["price"],
                        "implied_prob": 1.0 / float(outcome["price"]),
                    })
            elif market["key"] == "totals":
                for outcome in market["outcomes"]:
                    rows.append({
                        "fight": f"{event['home_team']} vs {event['away_team']}",
                        "commence_time": event["commence_time"],
                        "bookmaker": bookmaker["title"],
                        "fighter": f'{outcome["name"]} {outcome["point"]}',
                        "decimal_odds": outcome["price"],
                        "implied_prob": 1.0 / float(outcome["price"]),
                    })

df = pd.DataFrame(rows)
df['commence_time'] = pd.to_datetime(df['commence_time'])
print(df.head(10))

In [ ]:
df_old = df.copy()
# current date (UTC) + 30 days
cutoff = pd.Timestamp.now(tz="UTC") + pd.Timedelta(days=30)
df = df.loc[df['commence_time'] <= cutoff].reset_index(drop=True)

In [ ]:
import seaborn as sns

# Pivot so fighters are rows and bookmakers are columns
pivot_df = df.pivot_table(
    index=["fight", "fighter"],
    columns="bookmaker",
    values="decimal_odds"
)

plt.figure(figsize=(12,16))
sns.heatmap(pivot_df, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("MMA Fight Odds Across Bookmakers")
plt.ylabel("Fight / Fighter")
plt.xlabel("Bookmaker")
plt.show()


In [ ]:
# # 0) Setup
# from pathlib import Path

# PATH = Path("./odds_data/2025-09-02_mma_mixed_martial_arts.txt")  # your uploaded file


# # 1) Load -> tidy DataFrame (H2H only)
# def load_odds(path: str | Path) -> pd.DataFrame:
#     data = json.loads(Path(path).read_text(encoding="utf-8"))
#     rows = []
#     for ev in data:
#         fight = f"{ev['home_team']} vs {ev['away_team']}"
#         start = ev["commence_time"]
#         for bm in ev.get("bookmakers", []):
#             bm_name = bm.get("title") or bm.get("key")
#             for m in bm.get("markets", []):
#                 if m.get("key") != "h2h":
#                     continue
#                 for out in m.get("outcomes", []):
#                     rows.append({
#                         "fight": fight,
#                         "commence_time": start,
#                         "bookmaker": bm_name,
#                         "fighter": out["name"],
#                         "decimal_odds": float(out["price"]),
#                         "implied_prob": 1.0 / float(out["price"])
#                     })
#     df = pd.DataFrame(rows)
#     return df.sort_values(["fight", "bookmaker", "fighter"]).reset_index(drop=True)

# df = load_odds(PATH)
# df['commence_time'] = pd.to_datetime(df['commence_time'])

# # current date (UTC) + 30 days
# cutoff = pd.Timestamp.now(tz="UTC") + pd.Timedelta(days=30)
# df = df.loc[df['commence_time'] <= cutoff].reset_index(drop=True)

# df.head()


In [ ]:
# 2) Fix the earlier bar plotting issue by pivoting (prevents overwrites)
#    -> one grouped bar chart per fight, all fights automatically
def plot_all_fights_grouped(df: pd.DataFrame, value_col: str = "decimal_odds"):
    fights = df["fight"].unique()
    for fight in fights:
        sub = df[df["fight"] == fight].copy()
        # pivot: rows=fighter, cols=bookmaker
        pivot = sub.pivot_table(index="fighter", columns="bookmaker", values=value_col, aggfunc="first")
        # consistent bookmaker order
        bookmakers = list(pivot.columns)
        fighters = list(pivot.index)

        # grouped bars: x = bookmakers, 2 bars (or more) per bookmaker, one per fighter
        import numpy as np
        x = np.arange(len(bookmakers))
        width = 0.8 / max(2, len(fighters))  # spread bars across each bookmaker

        plt.figure(figsize=(10, 6))
        for i, fighter in enumerate(fighters):
            y = pivot.loc[fighter, bookmakers].values.astype(float)
            plt.bar(x + (i - (len(fighters)-1)/2)*width, y, width=width, label=fighter)

        yl = "Decimal Odds" if value_col == "decimal_odds" else "Implied Probability"
        plt.title(f"Odds across Books — {fight}")
        plt.xlabel("Bookmaker")
        plt.ylabel(yl)
        plt.xticks(x, bookmakers, rotation=35, ha="right")
        plt.legend()
        plt.tight_layout()
        plt.show()

# Example: decimal odds charts for all fights
plot_all_fights_grouped(df, value_col="decimal_odds")


In [ ]:
# Example: implied probabilities (0–1) for all fights
plot_all_fights_grouped(df, value_col="implied_prob")

In [ ]:
# 3) Optional: “best price by fighter” table (useful for line shopping)
def best_prices(df: pd.DataFrame) -> pd.DataFrame:
    idx = df.groupby(["fight", "fighter"])["decimal_odds"].idxmax()
    best = df.loc[idx, ["fight", "fighter", "decimal_odds", "bookmaker"]].sort_values(["fight","fighter"])
    best = best.rename(columns={"decimal_odds": "best_decimal", "bookmaker": "best_book"})
    return best.reset_index(drop=True)

best_prices(df)


In [ ]:
# 4) Optional: quick market snapshot per fight (favorite vs underdog, average prices)
def market_snapshot(df: pd.DataFrame) -> pd.DataFrame:
    # average decimal odds per fighter
    avg = (df.groupby(["fight", "fighter"])["decimal_odds"]
             .mean()
             .rename("avg_decimal")
             .reset_index())
    # favorite/underdog label within each fight
    avg["rank"] = avg.groupby("fight")["avg_decimal"].rank(method="first")
    avg["role"] = avg.groupby("fight")["avg_decimal"].transform(lambda s: ["favorite" if x==s.min() else "underdog" for x in s])
    return avg.sort_values(["fight","role","avg_decimal"]).reset_index(drop=True)

snap = market_snapshot(df)
snap


In [ ]:
df = df.merge(snap, left_on=['fight', 'fighter'], right_on=['fight', 'fighter'], how='outer')
df

In [ ]:
df.to_csv(f'./odds_data/{today_str}_{t_sport_keys[0]}.csv',index=False)

In [ ]:
df.to_clipboard()

In [ ]:
r2.headers['X-Requests-Used']